# Visual comparison of parameters for images in SAR and geographic coordinates

- This is only to check the outputs for geocoded images
- Numerical comparison is done in a script with C outputs using reference data.

In [1]:
%load_ext autoreload
%autoreload 2

import os

import numpy as np
from pathlib import Path
import xarray as xr
from polsarpro.io import open_netcdf_beam
from polsarpro.decompositions import h_a_alpha
from polsarpro.util import S_to_T3, boxcar
import matplotlib.pyplot as plt
# optional import for progress bar
from dask.diagnostics import ProgressBar

# change to your data paths
# original dataset
input_alos_slc = Path("/data/psp/test_files/SAN_FRANCISCO_ALOS1_slc.nc")
input_alos_geo = Path("/data/psp/test_files/SAN_FRANCISCO_ALOS1_geocoded_T3_7_look_az.nc")

# input files from C
input_test_dir = Path("/data/psp/test_files/input/h_a_alpha_decomposition/")

# output files from C
output_test_dir = Path("/data/psp/res/h_a_alpha_c")

In [2]:
slc = open_netcdf_beam(input_alos_slc)
geo = open_netcdf_beam(input_alos_geo)

# uncomment if you want to test for cropped data
# geo_crop = geo.isel(lat=slice(0,1000))

In [ ]:
# let's use persist (retains dask arrays) instead of compute (produces numpy arrays)
with ProgressBar():
    res_slc = h_a_alpha(slc, boxcar_size=[7 ,7]).persist()
with ProgressBar():
    # data has been multilooked in azimuth so we need less filtering
    res_geo = h_a_alpha(geo, boxcar_size=[3 ,3]).persist()

    # uncomment if you want to test for cropped data
    # res_geo = h_a_alpha(geo_crop, boxcar_size=[7 ,7]).compute()

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(res_slc.anisotropy[::8], interpolation="none")
plt.colorbar(fraction=0.046, pad=0.04)
plt.figure(figsize=(10, 10))
plt.imshow(res_geo.anisotropy, interpolation="none", vmin=0, vmax=1)
plt.colorbar(fraction=0.046, pad=0.04)

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(res_slc.alpha[::8], interpolation="none")
plt.colorbar(fraction=0.046, pad=0.04)
plt.figure(figsize=(10, 10))
plt.imshow(res_geo.alpha, interpolation="none", vmin=0, vmax=90)
plt.colorbar(fraction=0.046, pad=0.04)

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(res_slc.entropy[::8], interpolation="none")
plt.colorbar(fraction=0.046, pad=0.04)
plt.figure(figsize=(10, 10))
plt.imshow(res_geo.entropy, interpolation="none")
plt.colorbar(fraction=0.046, pad=0.04)